In [ ]:
'''
此為使用Selenium自動化爬取"網紅配方"官網全網紅的內頁連結
This code is for using Selenium to automate the web scraping of individual page links for all influencers on the "網紅配方" official website.
'''

In [ ]:
pip install fake_useragent
pip install webdriver_manager
pip install selenium

In [28]:
username = '123@gmail.com'  # type loggin username
pwd = '123'  # type loggin passwd

In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from time import sleep
import re
import numpy as np
import csv
import os
import random
import time

#檢查要放資料的目錄否存在
save_dir = './KOL_url'
if not os.path.exists(save_dir):
    print(f"放檔案的 {save_dir} 目錄不在，已建立{save_dir}")
    os.makedirs(save_dir)
else:
    print("KOL_url目錄已存在，將開始爬蟲")

# 生成介於180到300秒之間的隨機數
delay = random.randint(5, 10)   
    
#設定新視窗模式
my_options = webdriver.ChromeOptions()
my_options.add_argument("--start-maximized")        
my_options.add_argument("--incognito")              
my_options.add_argument("--disable-popup-blocking") 
my_options.add_argument("--disable-notifications")  
my_options.add_argument("--lang=zh-TW") 

#開啟google chrome，並套入上方寫入的模式
driver = webdriver.Chrome(
    options = my_options
)

def visit():
    url = "https://www.prefluencer.com/brands/login"
    driver.get(url)
    sleep(1)

def loggin():
    #輸入帳密
    account = driver.find_element(By.CSS_SELECTOR, "input#login_account")
    account.send_keys(username)
    sleep(1)
    password = driver.find_element(By.CSS_SELECTOR, "input#login_password")
    password.send_keys(pwd)
    sleep(1)
    #登入
    button = driver.find_element(By.CSS_SELECTOR, "button.btn.btn--block.btn--primary.btn--square.btn-login")
    button.click()
    sleep(3)
def scroll_and_click_next(scroll_times):
    spec_url = "https://www.prefluencer.com/brands"
    driver.get(spec_url)
    button = driver.find_element(By.CSS_SELECTOR, "button.btn.tour-guide__skip-btn")
    button.click()
    kol_names = []  # 初始化KOL名字列表
    content_links = []  # 初始化内页网址列表
    sleep(5)
    #-------------------------#
    #條件function可以放這裡
    #-------------------------#
    sleep(30)
    kol_names, content_links = desc_climb(kol_names, content_links)
    for _ in range(scroll_times):
        
        js = 'window.scrollTo(0, document.body.scrollHeight);'
        driver.execute_script(js)
        time.sleep(delay) 
        try:
            button = driver.find_element(By.CSS_SELECTOR, "a.btn.pagination_button.pagination_button-next")
            button.click()
            time.sleep(delay) 
        except Exception as e:
            print("無法找到下一頁按鈕或點擊失敗:", e)
            break
        kol_names, content_links = desc_climb(kol_names, content_links)
    return kol_names, content_links

def desc_climb(kol_names, content_links):
    soup = bs(driver.page_source, "lxml")

    kol_name_elements = soup.select('a.kollist__item-link--title')
    content_link_elements = soup.select('a[class="kollist__item-link--title text-ellipsis"]')

    # 循環處理每個匹配的元素
    for kol_name_element in kol_name_elements:
        kol_names.append(kol_name_element.text)
    # 刪除最後的奇怪元素
    if kol_names:
        kol_names = [name for name in kol_names if name != '{name}']

    for content_link_element in content_link_elements:
        url = f'"https://www.prefluencer.com{content_link_element["href"]}",'
        # 删除以 'https://www.prefluencer.com{url}' 开头的 URL
        if not url.startswith('https://www.prefluencer.com{url}'):
            content_links.append(url)
    
    return kol_names, content_links

# 以下是你的主程序
if __name__ == "__main__":
    visit()
    loggin()
    kol_names, formatted_content_links = scroll_and_click_next(97)

print("爬蟲已完成")
print("正在寫入檔案...")

# 寫入CSV文件
try:
    csv_file_path = os.path.join(save_dir, f"KOL內頁網址1K-3K.csv")
    with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ['KOL名字', '內頁網址']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(kol_names)):
            writer.writerow({'KOL名字': kol_names[i], '內頁網址': formatted_content_links[i]})
    print(f"CSV儲存成功，腳本已結束")

except Exception as e:
    print(f"CSV時發生錯誤: {e}")

KOL_url目錄已存在，將開始爬蟲
爬蟲已完成
正在寫入檔案...
CSV儲存成功，腳本已結束


In [33]:
#選擇300k以上網紅
def follower_300k_up():
    button = driver.find_element(By.CSS_SELECTOR, "label[for='follower_5']")
    button.click()

In [4]:
#選擇150k~300k網紅
def follower_150k_300k():
    button = driver.find_element(By.CSS_SELECTOR, "label[for='follower_4']")
    button.click()

In [20]:
#選擇50k~150k網紅
def follower_50k_150k():
    button = driver.find_element(By.CSS_SELECTOR, "label[for='follower_3']")
    button.click()

In [10]:
#選擇10k~50k網紅
def follower_10k_50k():
    button = driver.find_element(By.CSS_SELECTOR, "label[for='follower_2']")
    button.click()

In [11]:
#選擇1k~10k網紅
def follower_1k_10k():
    button = driver.find_element(By.CSS_SELECTOR, "label[for='follower_1']")
    button.click()

In [ ]:
def srh_bar():
    target_min = 3000
    target_max = 5000
    slider = driver.find_element(By.CSS_SELECTOR, "div.input-slider")
    driver.execute_script(f"arguments[0].setAttribute('data-slider_value', {target_value});", slider)
    driver.execute_script("var event = new Event('change'); arguments[0].dispatchEvent(event);", slider)

In [32]:
import pandas as pd

# 讀取CSV文件
df = pd.read_csv('./KOL_url/KOL內頁網址300K.csv')

# 去除網址欄位中的雙引號
df['內頁網址'] = df['內頁網址'].str.strip('"')

# 將結果保存回CSV文件
df.to_csv('./KOL_url/KOL內頁網址300K.csv', index=False)